# Converter for the Financial Filings at the Sec

In [51]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2011q1.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t")
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2011q1"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

6374628
2619880
54000
248634
24168


In [52]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+-------------------------------------------------------+--------------+-------------------------+----------+--------+-------+--------------+------------+
|    | adsh                 | tag                                                   | version      | coreg                   |    ddate |   qtrs | uom   |        value |   footnote |
|----+----------------------+-------------------------------------------------------+--------------+-------------------------+----------+--------+-------+--------------+------------|
|  0 | 0001193125-11-038642 | AcceleratedShareRepurchasesFinalPricePaidPerShare     | us-gaap/2009 | nan                     | 20091231 |      0 | USD   | 39.53        |        nan |
|  1 | 0000930413-11-001115 | AcceleratedShareRepurchasesFinalPricePaidPerShare     | us-gaap/2009 | nan                     | 20100331 |      1 | USD   | 56.21        |        nan |
|  2 | 0000930413-11-001115 | AcceleratedShareRepurchasesInitialPricePaidPerShare   |

In [53]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+----------+--------+--------+---------+---------+---------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                               | version      | plabel                                                                                                                                                  |   negating |
|----+----------------------+----------+--------+--------+---------+---------+---------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+------------|
|  0 | 0000890319-11-000009 |        1 |     13 | BS     |   

In [54]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------+------------------------------------------+-------+-------------+----------+----------------------+------------+------------------------------------------+---------------------------+-------------------+-------------+----------+----------------------------+------------+------------------------------------------+----------------------------+--------------+-----------+-----------+----------------------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+--------------------+---------+---------------------------------------------------------------------------------------------------------+
|    | adsh                 |     cik | name                                     |   sic | countryba   | stprba   | cityba               | zipba      | bas1                                     | bas2                      | baph              | countryma   |

In [55]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))

+----+-------------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+-------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+


In [57]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                                     0001193125-11-038642
tag         AcceleratedShareRepurchasesFinalPricePaidPerShare
version                                          us-gaap/2009
coreg                                                     NaN
ddate                                                20091231
qtrs                                                        0
uom                                                       USD
value                                                   39.53
footnote                                                  NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [67]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

# Create a single SymbolFinancialsDto

In [ ]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    sfDto.symbol = val["symbol"].to_string(index = False).upper()
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myDto.info = myPre["plabel"].to_string(index = False)
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic)))
    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(str(result))
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("file time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec.")

FilterDfNum size: 281 Dtos size: 185 Bs: 105 Cf: 80 Ic: 0
Json size 36kb. Time: 2.306ms.
file 1 exportFiles/2011q1/0000930413-11-002462.json stored in 5369.589ms.
FilterDfNum size: 395 Dtos size: 239 Bs: 102 Cf: 137 Ic: 0
Json size 50kb. Time: 1.913ms.
file 2 exportFiles/2011q1/0000950123-11-031061.json stored in 7543.451ms.
FilterDfNum size: 1852 Dtos size: 568 Bs: 319 Cf: 249 Ic: 0
Json size 116kb. Time: 4.184ms.
file 3 exportFiles/2011q1/0000950123-11-031283.json stored in 34878.072ms.
FilterDfNum size: 939 Dtos size: 169 Bs: 86 Cf: 83 Ic: 0
Json size 37kb. Time: 1.524ms.
file 4 exportFiles/2011q1/0001104659-11-017950.json stored in 17589.402ms.
FilterDfNum size: 254 Dtos size: 151 Bs: 59 Cf: 92 Ic: 0
Json size 33kb. Time: 1.512ms.
file 5 exportFiles/2011q1/0001178913-11-000967.json stored in 4876.962ms.
FilterDfNum size: 670 Dtos size: 158 Bs: 59 Cf: 99 Ic: 0
Json size 34kb. Time: 1.325ms.
file 6 exportFiles/2011q1/0001193125-11-084306.json stored in 11725.605ms.
FilterDfNum size: 